In [1]:
(require booleans)

Some possibly helpful laws:

Posulate 1: `¬( GO ∨ (RES ∧ SEL))` implies all `E'` is `0` and all `kn` are `0`.
  Correlary 1: This holds for `¬(GO ∨ SEL)`
  Correlary 2: This holds for `¬(GO ∨ RES)`

Postulate 2: forall `n` `m`, `m != n`, `¬ (kn ^ km)`, that is at most one `k` wire is selected.


In [2]:
(define/ppl start #:no-check
  ((K0 = (and left (and right both)))
   (left = (or l0 lem))
   (right = (or r0 rem))
   (l0 = GO)
   (lem = (not (or GO false)))
   (rem = (not (or GO sel)))
   (both = (or l0 r0))))

(K0 = (and left (and right both)))
(left = (or l0 lem))
(right = (or r0 rem))
(l0 = GO)
(lem = (not (or GO false)))
(rem = (not (or GO sel)))
(both = (or l0 r0))


In [3]:
(define/ppl translated #:no-check
  (convert-P start))

((+ K0) = (and (+ left) (and (+ right) (+ both))))
((- K0) = (or (- left) (or (- right) (- both))))
((+ left) = (or (+ l0) (+ lem)))
((- left) = (and (- l0) (- lem)))
((+ right) = (or (+ r0) (+ rem)))
((- right) = (and (- r0) (- rem)))
((+ l0) = (+ GO))
((- l0) = (- GO))
((+ lem) = (and (- GO) true))
((- lem) = (or (+ GO) false))
((+ rem) = (and (- GO) (- sel)))
((- rem) = (or (+ GO) (+ sel)))
((+ both) = (or (+ l0) (+ r0)))
((- both) = (and (- l0) (- r0)))


Step 1: By rules of `and` and `or`

In [4]:
(define/ppl step1
  replace* translated
           [(or (+ GO) false) (+ GO)]
           [(and (- GO) true) (- GO)])

((+ K0) = (and (+ left) (and (+ right) (+ both))))
((- K0) = (or (- left) (or (- right) (- both))))
((+ left) = (or (+ l0) (+ lem)))
((- left) = (and (- l0) (- lem)))
((+ right) = (or (+ r0) (+ rem)))
((- right) = (and (- r0) (- rem)))
((+ l0) = (+ GO))
((- l0) = (- GO))
((+ lem) = (- GO))
((- lem) = (+ GO))
((+ rem) = (and (- GO) (- sel)))
((- rem) = (or (+ GO) (+ sel)))
((+ both) = (or (+ l0) (+ r0)))
((- both) = (and (- l0) (- r0)))


In [5]:
(define/ppl step2
  propigate/remove* step1 (+ l0) (- l0) (+ lem) (- lem))

((+ K0) = (and (+ left) (and (+ right) (+ both))))
((- K0) = (or (- left) (or (- right) (- both))))
((+ left) = (or (+ GO) (- GO)))
((- left) = (and (- GO) (+ GO)))
((+ right) = (or (+ r0) (+ rem)))
((- right) = (and (- r0) (- rem)))
((+ rem) = (and (- GO) (- sel)))
((- rem) = (or (+ GO) (+ sel)))
((+ both) = (or (+ GO) (+ r0)))
((- both) = (and (- GO) (- r0)))


By +x and -x = 0 and law of the excluded middle we can have
three cases:

1. `+GO` = `-GO` = 0
2. `+GO` = 1, `-GO` = 0
3. `+GO` = 0, `-GO` = 1

# CASE 1 `+GO` = `-GO` = 0 (`GO` is unknown)

In [6]:
(define/ppl case1-step1 #:no-check
  (replace* 
   step2
   [(+ GO) false]
   [(- GO) false]))

((+ K0) = (and (+ left) (and (+ right) (+ both))))
((- K0) = (or (- left) (or (- right) (- both))))
((+ left) = (or false false))
((- left) = (and false false))
((+ right) = (or (+ r0) (+ rem)))
((- right) = (and (- r0) (- rem)))
((+ rem) = (and false (- sel)))
((- rem) = (or false (+ sel)))
((+ both) = (or false (+ r0)))
((- both) = (and false (- r0)))


by the laws of and/or on false

In [7]:
(define/ppl case1-step2
  replace* case1-step1
           [(and false (- sel)) false]
           [(or false (+ sel))  (+ sel)]
           [(or false (+ r0)) (+ r0)]
           [(and false (- r0)) false]
           [(or false false) false]
           [(and false false) false])

((+ K0) = (and (+ left) (and (+ right) (+ both))))
((- K0) = (or (- left) (or (- right) (- both))))
((+ left) = false)
((- left) = false)
((+ right) = (or (+ r0) (+ rem)))
((- right) = (and (- r0) (- rem)))
((+ rem) = false)
((- rem) = (+ sel))
((+ both) = (+ r0))
((- both) = false)


In [8]:
(define/ppl case1-step3
   propigate/remove* case1-step2
                     (+ rem) (- rem)
                     (+ both) (- both)
                     (+ left) (- left))

((+ K0) = (and false (and (+ right) (+ r0))))
((- K0) = (or false (or (- right) false)))
((+ right) = (or (+ r0) false))
((- right) = (and (- r0) (+ sel)))


In [9]:
(define/ppl case1-step4
   replace* case1-step3
            [(or (+ r0) false) (+ r0)]
            [(and false (and (+ right) (+ r0))) false]
            [(or false (or (- right) false)) (- right)])

((+ K0) = false)
((- K0) = (- right))
((+ right) = (+ r0))
((- right) = (and (- r0) (+ sel)))


In [10]:
(define/ppl case1-step5
            propigate/remove* case1-step4
                              (+ right)
                              (- right))

((+ K0) = false)
((- K0) = (and (- r0) (+ sel)))


By Postulate 1 Correlay 1, `r0` is only set if we have either `GO` or `SEL`, Since we don't have `GO` here, we can only have `R0` if `+sel` is `1`.  Thus this reads: if `SEL` is undefined or not set, then `k0` is undefined. If `sel` is defined and `1` then `k0` is `0` if `r0` is `0`, and undefined otherwise. This last part seems wrong...

# CASE 2 `+GO` = 1 and `-GO` = 0 (`GO` is 1)

In [11]:
(define/ppl case2-step1 #:no-check
  (replace* step2
            [(+ GO) true]
            [(- GO) false]))

((+ K0) = (and (+ left) (and (+ right) (+ both))))
((- K0) = (or (- left) (or (- right) (- both))))
((+ left) = (or true false))
((- left) = (and false true))
((+ right) = (or (+ r0) (+ rem)))
((- right) = (and (- r0) (- rem)))
((+ rem) = (and false (- sel)))
((- rem) = (or true (+ sel)))
((+ both) = (or true (+ r0)))
((- both) = (and false (- r0)))


In [12]:
(define/ppl case2-step2
             replace* case2-step1
                      [(or true false) true]
                      [(and false true) false]
                      [(and false (- sel)) false]
                      [(or true (+ sel)) true]
                      [(or true (+ r0)) true]
                      [(and false (- r0)) false])

((+ K0) = (and (+ left) (and (+ right) (+ both))))
((- K0) = (or (- left) (or (- right) (- both))))
((+ left) = true)
((- left) = false)
((+ right) = (or (+ r0) (+ rem)))
((- right) = (and (- r0) (- rem)))
((+ rem) = false)
((- rem) = true)
((+ both) = true)
((- both) = false)


In [13]:
(define/ppl case2-step3
             propigate/remove* case2-step2
                               (+ left) (- left)
                               (+ rem) (- rem)
                               (+ both) (- both))

((+ K0) = (and true (and (+ right) true)))
((- K0) = (or false (or (- right) false)))
((+ right) = (or (+ r0) false))
((- right) = (and (- r0) true))


In [14]:
(define/ppl case2-step4
             replace* case2-step3
                      [(and true (and (+ right) true))
                       (+ right)]
                      [(or false (or (- right) false))
                       (- right)]
                      [(or (+ r0) false) (+ r0)])

((+ K0) = (+ right))
((- K0) = (- right))
((+ right) = (+ r0))
((- right) = (and (- r0) true))


In [15]:
(define/ppl case2-step5 
             propigate/remove* case2-step4
                               (+ right)
                               (- right))

((+ K0) = (+ r0))
((- K0) = (and (- r0) true))


In [16]:
(define/ppl case2-step6
             replace* case2-step5
                      [(and (+ r0) (+ r0)) (+ r0)]
                      [(and (- r0) true) (- r0)])

((+ K0) = (+ r0))
((- K0) = (- r0))


# CASE 3 `+GO` = 0 and `-GO` = 1 (`GO` is 0)

In [17]:
(define/ppl case3-step1 #:no-check
   (replace* step2
            [(+ GO) false]
            [(- GO) true]))

((+ K0) = (and (+ left) (and (+ right) (+ both))))
((- K0) = (or (- left) (or (- right) (- both))))
((+ left) = (or false true))
((- left) = (and true false))
((+ right) = (or (+ r0) (+ rem)))
((- right) = (and (- r0) (- rem)))
((+ rem) = (and true (- sel)))
((- rem) = (or false (+ sel)))
((+ both) = (or false (+ r0)))
((- both) = (and true (- r0)))


In [18]:
(define/ppl case3-step2
             replace* case3-step1
                      [(or false true) true]
                      [(and true false) false]
                      [(and true (- sel)) (- sel)]
                      [(or false (+ sel)) (+ sel)]
                      [(or false (+ r0)) (+ r0)]
                      [(and true (- r0)) (- r0)])

((+ K0) = (and (+ left) (and (+ right) (+ both))))
((- K0) = (or (- left) (or (- right) (- both))))
((+ left) = true)
((- left) = false)
((+ right) = (or (+ r0) (+ rem)))
((- right) = (and (- r0) (- rem)))
((+ rem) = (- sel))
((- rem) = (+ sel))
((+ both) = (+ r0))
((- both) = (- r0))


In [19]:
(define/ppl case3-step3
             propigate/remove* case3-step2
                               (+ left) (- left)
                               (+ rem) (- rem)
                               (+ both) (- both))

((+ K0) = (and true (and (+ right) (+ r0))))
((- K0) = (or false (or (- right) (- r0))))
((+ right) = (or (+ r0) (- sel)))
((- right) = (and (- r0) (+ sel)))


In [20]:
(define/ppl case3-step4
             replace* case3-step3
             [(and true (and (+ right) (+ r0)))
              (and (+ right) (+ r0))]
             [(or false (or (- right) (- r0)))
                  (or (- right) (- r0))])

((+ K0) = (and (+ right) (+ r0)))
((- K0) = (or (- right) (- r0)))
((+ right) = (or (+ r0) (- sel)))
((- right) = (and (- r0) (+ sel)))


In [21]:
(define/ppl case3-step5
             propigate/remove*
             case3-step4
             (+ right) (- right))

((+ K0) = (and (or (+ r0) (- sel)) (+ r0)))
((- K0) = (or (and (- r0) (+ sel)) (- r0)))


Again, we use the law of excluded middle on `+r0` to check against 0 and 1. If we do this we can see that `(and (or (+ r0) (- sel)) (+ r0))` is the same as `(+ r0)`.

In [22]:
(define/ppl case3-step6
             replace*
             case3-step5
             [(and (or (+ r0) (- sel)) (+ r0))
              (+ r0)])

((+ K0) = (+ r0))
((- K0) = (or (and (- r0) (+ sel)) (- r0)))


We can also do the same with `-r0`, and see that we get a result equal to `-r0` in `-K0` for either value.

In [23]:
(define/ppl case3-step7
             replace*
             case3-step6
             [(or (and (- r0) (+ sel)) (- r0))
              (- r0)])

((+ K0) = (+ r0))
((- K0) = (- r0))


## QED